In [1]:
##With preprocessed RFC-Discussion getting Information on Users
import pandas as pd
import json
import re

wikipedia_parsed = "../json_files/grawitas_output/wikipedia_parsed.json"
wikidata_parsed = "../json_files/grawitas_output/wikidata_parsed.json"
meta_parsed = "../json_files/grawitas_output/meta_parsed.json"
with open(wikipedia_parsed) as f:
        wikipedia_list_of_dicts = json.load(f)
with open(wikidata_parsed) as f:
        wikidata_list_of_dicts = json.load(f)
with open(meta_parsed) as f:
        meta_list_of_dicts = json.load(f)
        
def getcleanDataFrame(json):
    df = pd.json_normalize(json, "page_text", ["page_title"])
    #remove IP-Adresses from userArray
    df2 = df[~df['user'].str.contains(re.compile(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b'))]
    return list(df2["user"].unique())


wikipedia_user_list = getcleanDataFrame(wikipedia_list_of_dicts)
wikidata_user_list = getcleanDataFrame(wikidata_list_of_dicts)
meta_user_list = getcleanDataFrame(wikidata_list_of_dicts)

# Concatenate the lists
merged_user_list = wikipedia_user_list + wikidata_user_list + meta_user_list

# Convert to a set to remove duplicates
unique_user_list = list(set(merged_user_list))

In [2]:
print(len(unique_user_list))

8037


In [3]:
from userinformation import getUserInfoToJSON
getUserInfoToJSON(unique_user_list, "../json_files/user_info/users.json")

In [4]:
users = "../json_files/user_info/users.json"
with open(users) as f:
        user_list_of_dicts = json.load(f)
df = pd.json_normalize(user_list_of_dicts)

In [11]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8037 entries, 0 to 8036
Data columns (total 36 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   user_id                                      6466 non-null   float64
 1   user_name                                    8037 non-null   object 
 2   user_real_name                               6466 non-null   object 
 3   seconds_between_regdate_and_first_edit_date  7304 non-null   float64
 4   isDeleted                                    8037 non-null   bool   
 5   user_registration                            7329 non-null   object 
 6   editcount.mediawikiwiki                      7329 non-null   float64
 7   editcount.metawiki                           7329 non-null   float64
 8   editcount.wikidatawiki                       7329 non-null   float64
 9   editcount.enwiki                             7329 non-null   float64
 10  

,user_id,user_name,user_real_name,seconds_between_regdate_and_first_edit_date,isDeleted,user_registration,editcount.mediawikiwiki,editcount.metawiki,editcount.wikidatawiki,editcount.enwiki,...,permissions.dewiki,permissions.zhwiki,permissions.jawiki,permissions.plwiki,permissions.ruwiki,permissions.itwiki,permissions.nlwiki,permissions.ptwiki,editcount,permissions
0,31965.0,Soulkeeper,,178922.0,False,2006-01-25T23:59:23,3.0,48.0,189695.0,2808.0,...,user,user,user,user,user,user,user,user,NaN,NaN
1,136141.0,Floquenbeam,,824.0,False,2008-08-13T19:02:38,0.0,23.0,13.0,36546.0,...,user,user,user,user,user,user,user,user,NaN,NaN
2,10475519.0,"Quick, Spot the Quetzalcoatl!",,1636.0,False,2017-06-27T16:30:56,0.0,0.0,1.0,2208.0,...,user,user,user,user,user,user,user,user,NaN,NaN
3,2021459.0,JoelleJay,,12968939.0,False,2007-06-27T04:23:02,0.0,0.0,8.0,6528.0,...,user,user,user,user,user,user,user,user,NaN,NaN
4,NaN,Joyous%21,None,NaN,True,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
